# Computer Science 2XC3 - Graded Lab II

Please refer to the pdf for detailed instructions. The below file contains all the preliminary code you will need to work on the lab. You can copy paste instructions here to create one cohesive lab and organize it that best suits your teams workflow. 

In [2]:
import random
import timeit 
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
class GraphI:

    # using hash map
    def __init__(self, edges):
        self.graph = {}
        for x,y in edges:
            if x not in self.graph.keys():
                self.graph[x]=[]
            self.graph[x].append(y)

    def has_edge(self, src, dst):
        return src in self.graph[dst]

    def get_graph_size(self,):
        return len(self.graph)
    
    def get_graph(self,):
        return self.graph

In [4]:
class GraphII:

    # using adjacency list
    def __init__(self, nodes):
        self.graph = []
        # node numbered 0-1
        for node in range(nodes):
            self.graph.append([])
        
    def has_edge(self, src, dst):
        return src in self.graph[dst]
    
    def add_edge(self,src,dst):
        if not self.has_edge(src,dst):
            self.graph[src].append(dst)
            self.graph[dst].append(src)
    
    def get_graph(self,):
        return self.graph

In [5]:
def depth_first_search(G,node,end_point=None):
    stack = [node]
    graph = G.get_graph()
    seen=set()

    while len(stack) !=0:
        node = stack.pop()
        # search for neighbours in graph
        if node not in seen:
            seen.add(node)
            print("Visited node:" + str(node))
            # if the given node has an edge
            if node in graph.keys():
                # iterate over edges of node
                for nn in graph[node]: 

                    # limited traversal
                    if nn == end_point:
                        return True
                    # add to stack
                    stack.append(nn)

In [6]:
#Breadth First Search
def breadth_first_search(G, node):
    stack = [node]
    graph = G.get_graph()
    seen=set()

    seen.add(node)

    while len(stack) > 0:
        node = stack[0]
        stack = stack[1:]
        print("Visiting node: " + str(node))
        if node in graph.keys():
            for nn in graph[node]:
                #if node == node2:
                #    return True
                if nn not in seen:
                    stack.append(nn)
                    seen.add(nn)

In [7]:
#Use the methods below to determine minimum vertex covers

def add_to_each(sets, element):
    copy = sets.copy()
    for set in copy:
        set.append(element)
    return copy

def power_set(set):
    if set == []:
        return [[]]
    return power_set(set[1:]) + add_to_each(power_set(set[1:]), set[0])

def is_vertex_cover(G, C):
    for start in G.adj:
        for end in G.adj[start]:
            if not(start in C or end in C):
                return False
    return True

def MVC(G):
    nodes = [i for i in range(G.get_size())]
    subsets = power_set(nodes)
    min_cover = nodes
    for subset in subsets:
        if is_vertex_cover(G, subset):
            if len(subset) < len(min_cover):
                min_cover = subset
    return min_cover


In [8]:
# test_graph = GraphII(4)
# test_graph.add_edge(0, 1)
# test_graph.add_edge(0, 2)
# test_graph.add_edge(0, 3)
# test_graph.add_edge(2, 3)

# test_graph.get_graph()

# 0 has edges 1, 2, 3. 1 has edge 0, 2 has edge 0 and 3, 3 has edge 0 and 2

test_graph = GraphI([[0, 1], [0, 2], [0, 3], [2, 3]])
test_graph.get_graph()
# breadth_first_search(test_graph, 0)

{0: [1, 2, 3], 2: [3]}

Part 1.1: Implement BFS2 and DFS2 where the path between two nodes node1 and node2 is returned as a
list. For instance, in a graph, if to reach node 8 from node 6, one needs to traverse the path starting at 6 to
23, to 12, then to 5, then to 10, and finally to 8, your function BFS2(graph, 6,8) (or DFS2(graph, 6,8) )
should return a list [6,23,12,5,10,8]. Implement both BFS2 and DFS2 for this variation.

In [9]:
# Using the given BFS code and modifying it

def bfs_2(G, src, dst):
    
    path_list = []

    stack = [src]
    graph = G.get_graph()
    seen = set()

    seen.add(src)
    
    path_list.append(src)

    while len(stack) > 0:
        node = stack[0]
        stack = stack[1:]
        # print("Visiting node: " + str(node))

        # Append node to new list if it hasn't been visited before
        if node not in path_list:
            path_list.append(node)
            # Return the list of nodes visited if the destination node is reached
            if node == dst:
                return path_list
        
        if node in graph.keys():
            for new_node in graph[node]:
                if new_node not in seen:
                    stack.append(new_node)
                    seen.add(new_node)
    return path_list
    
# Testing
# test_graph = GraphI([[0, 1], [1, 5], [1, 6], [5, 3], [5, 2], [5, 6], [6, 4], [6, 7]])
# test_graph.get_graph()
# # 0, 1, 5, 6, 2, 3, 4, 7
# bfs_2(test_graph, 0, 2)



In [10]:
# Using the given DFS code and modifying it

def dfs_2(G, src, dst, end_point=None):
    stack = [src]
    path_list = []
    graph = G.get_graph()
    seen = set()

    

    while len(stack) != 0:
        node = stack.pop()

        if node not in seen:
            seen.add(node)
            # print("Visited node: " + str(node))

            # Check if the visited node is already in path_list, and append it if not
            if node not in path_list:
                path_list.append(node)
                if node == dst:
                    return path_list

            if node in graph.keys():
                for new_node in graph[node]:
                    # Due to the given algorithm's limited implementation, it's possible for a dst node to be given that is in the graph but unreachable by the algorithm
                    if new_node == end_point:
                        return True
                    stack.append(new_node)

# Testing
# test_graph = GraphI([[1, 2], [2, 6], [6, 5], [6, 4]])
# dfs_2(test_graph, 2, 4)


Part 1.2:  In some applications, we need to find connections from a given node to all nodes. Think about
how one might find recommendations for possible connections on social media platforms. In this
variation implement BFS3 and DFS3 which take as an input 1 node and return paths to every other node
(note that this is different from all paths between all nodes. Your goal is to find a path to a node). These
paths should be returned as a “predecessor dictionary”. Predecessor dictionary contains the key as the
node and the value as the predecessor node. For example, for the following graph, your implementation of
BFS3(graph, 1) will return the predecessor dictionary as: {2 : 1, 3 : 1, 4 : 2, 5 : 3, 6 : 4}

In [41]:
#Using the given Breadth First Search algorithm and modifying it
def bfs_3(G, node):
    # Initializing predecessor dictionary
    pred_dict = {}
    graph = G.get_graph()

    stack = [node]
    seen=set()

    seen.add(node)


    while len(stack) > 0:
        
        node = stack[0]
        stack = stack[1:]
        print("Visiting node: " + str(node))
        

        if node in graph.keys():
            for new_node in graph[node]:

                #if node == node2:
                #    return True
                if new_node not in seen:
                    # If this node is new, it can be updated with the current node that's being iterated over
                    pred_dict[new_node] = node
                    stack.append(new_node)
                    seen.add(new_node)

    return pred_dict

# Testing
test_graph = GraphI([[1, 2], [1, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6]])
# test_graph.get_graph()
bfs_3(test_graph, 1)



Visiting node: 1
Visiting node: 2
Visiting node: 3
Visiting node: 4
Visiting node: 5
Visiting node: 6


{2: 1, 3: 1, 4: 2, 5: 3, 6: 4}

In [42]:
# Using the given Depth First Search Algorithm and modifying it
def dfs_3(G,node,end_point=None):
    # Initializing predecessor dictionary
    pred_dict = {}
    stack = [node]
    graph = G.get_graph()
    seen=set()


    while len(stack) !=0:
        node = stack.pop()
        # search for neighbours in graph
        if node not in seen:
            seen.add(node)
            print("Visited node:" + str(node))
            # if the given node has an edge
            if node in graph.keys():
                # iterate over edges of node
                for new_node in graph[node]: 
                    # Similar to bfs_3 implementation:
                    if new_node not in pred_dict:
                        pred_dict[new_node] = node
                    # limited traversal
                    if new_node == end_point:
                        return True
                    # add to stack
                    stack.append(new_node)
    
    return pred_dict

# Testing
# test_graph = GraphI([[1, 2], [1, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6]])
# test_graph.get_graph()
# {1: [2, 3], 2: [4], 3: [4, 5], 4: [6], 5: [4]}

# dfs_3(test_graph, 1)

Part 1.3: Implement a function in the graph class called has_cycle( ) that computes and returns True if
the graph has a cycle.

In [47]:
# Since the depth_first_search algorithm given in the lab has a default "endpoint" parameter,
# We can use the DFS function on the node to see if there is a reachable path from that same node
def has_cycle(G):
    graph = G.get_graph()

    # Iterate through each node, calling the given DFS with the startpoint and endpoint parameter as that node
    for node in graph:
        result = depth_first_search(G, node, node)
        if result == True:
            return True
    
    # If all nodes in the graph return false from the DFS, it must mean that no cycles exist
    return False

# Testing
# test_graph = GraphI([[0, 1], [1, 2], [2, 3]])
# has_cycle(test_graph)

# test_graph = GraphI([[0, 1], [1, 2], [2, 0], [2, 3]])
# has_cycle(test_graph)

Part 1.4: Implement a function in graph class called is_connected( ) that computes and returns True if
there is a path between two nodes. Note that this is different from what we discussed in class ( has_edge()
). While has_edge finds whether an edge exists between two nodes, is_connected finds whether there is a
path between two nodes.
Feel free to use any part of the code here.

In [50]:
# Similar to Part 1.3, we can run DFS to check if a path can be found between the given startpoint and endpoint
def is_connected(G, startpoint, endpoint):
    graph = G.get_graph()

    for node in graph:
        value = depth_first_search(G, startpoint, endpoint)
        if value == True:
            return True
    
    return False

# Testing: Used graph example from Part 1.2, except added some disconnected nodes
# test_graph = GraphI([[1, 2], [1, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [7, 8]])
# is_connected(test_graph, 1, 8)

Part 1.5: In the previous lab we conducted a few experiments using a random list generator that I
provided. What would that look like for a graph? To experiment with graphs, you want to be able to
generate random graphs. Write a function to do so. The way to approach this is to think about the
essential elements of the graph nodes (n) and edges (e). So when you call the function
create_random_graph(n,e), it should create a random layout with only a single edge between two nodes.

In [52]:
import random

def random_graph(num_nodes: int, num_edges: int):
    # nodes_arr = [x for x in range(num_nodes)]
    random_edges = []

    for i in range(num_edges):
        node_1 = random.randint(0, num_nodes)    
        node_2 = random.randint(0, num_nodes)

        if node_1 != node_2 and [node_1, node_2] not in random_graph:
            random_edges.append([node_1, node_2])

    random_graph = GraphI(random_edges) # put parameters here
    return random_graph

# random_graph(5, 6)

[0, 1, 2, 3, 4]